<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# CeRAS Case Study

This tutorial aims to retrieve CeRAS report results through a step by step approach. More information about the CeRAS reference aircraft configuration can be found [here](https://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/AircraftDesigns/CSR01). The objective is to retrieve the reference aircraft geometry and weights. 

Particularly, the Maximum Take-Off Weight (MTOW) target is 77000kg and the targetted Operational Weight Empty (OWE) is 42100kg.  

***
***
## Imports


In [ ]:
import os.path as pth
import sys
import logging
import shutil

import openmdao.api as om

import fastoad.api as oad

sys.path.append(pth.abspath("."))
from ceras_utils import plot_mission_against_ceras

## Working directories and reference files 
Here you can define your working directory and data storage folder as well as reference files names. 

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

DATA_FOLDER_PATH = "data"

WORK_FOLDER_PATH = "workdir"

SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "CeRAS_reference_data.xml")

REFERENCE_FILE_FOR_PLOT = pth.join(DATA_FOLDER_PATH, "CeRAS_data_for_plots.xml")

CERAS_REFERENCE_RESULTS_FILE = pth.join(
    DATA_FOLDER_PATH, "CSR-01_missionDesign_R4630_PL17000_out.csv"
)

CERAS_SPP_DESIGN_REFERENCE_RESULTS_FILE = pth.join(
    DATA_FOLDER_PATH, "CSR-01_missionStudy_R5093_PL13608_out.csv"
)

CERAS_SPP_STUDY_REFERENCE_RESULTS_FILE = pth.join(
    DATA_FOLDER_PATH, "CSR-01_missionStudy_R926_PL13608_out.csv"
)

## 1. OAD Process
For the first part of the exercice you will see how to retrieve CeRAS reference aircraft results from using the OAD process. 

The reference mission characteristics used for the sizing correspond to the *"MTOW mission"* defined in the CeRAS report whose specifications are reminded below: 

* Range = 2500NM
* Diversion range = 200NM
* Payload = 17000kg
* Taxi in = 5min 
* Taxi out = 9min 
* Cruise Mach = 0.78

### 1.1. The mission file

You will use the mission module to compute the fuel consumption over a specified mission. Therefore, you need to define the different phases your mission is composed of. That operation is done through the YAML [mission file](./data/CeRAS_missions.yml). If you have a closer look to this file, you will notice that it starts with a definition of the different flight phases (initial climb, climb, descent, holding, taxi) for which one or several flight parameter(s) is (are) varying.

For exemple, for the initial climb phase, the first segment is performed at iso-speed with a variation of altitude up to 400ft, then a second segment defines an acceleration at iso-altitude up to 250kt and a third segment at iso-speed and targetting an altitude of 15000ft completes this phase.

Each phases are then organised and completed with a cruise phase to form a route. Several routes can then be combined to form a mission profile. 

Each mission profile should have a unique name.

For further information about the mission definition, please see the [official documentation](https://fast-oad.readthedocs.io/en/documentation/mission_module/index.html).

### 1.2. The configuration file

As already mentioned, the [configuration file](./data/oad_sizing.yml) you are using enables you run the full OAD sizing process. Here a particular attention is paid on the performance module definition. 

First, in this section, you will call the [mission file](./data/CeRAS_missions.yml) you want to use through the `mission_file_path` option. You will then indicate which mission profile to select accordingly to the naming convention you chose in the mission file. This is done using the `mission_name` option. In the present situation you will compute performances for the mission called `MTOW_mission`. 

Note also that the option `is_sizing` of the performance group is active. That means that the mission TOW computed by the performance module is considered as the design Maximum Take-Off Weight and will impact the aircraft design. 

You can also notice that the option `add_solver` is deactivated as solvers are already defined for the whole problem in the [configuration file](./data/oad_sizing.yml).

In [ ]:
OAD_CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, "oad_sizing.yml")

### 1.3. The input file
You can now generate the input file and check the correctness of the TLARs and mission settings with respect to the reference mission specifications.

Particularly you should pay attention to the following inputs:

* `data:mission:MTOW_mission:main_route:range` = 2500NM
* `data:mission:MTOW_mission:diversion:distance` = 200NM
* `data:mission:MTOW_mission:taxi_in:duration` = 300s


Note that the Taxi out phase is not characterised by a duration like in CeRAS report specification but through a fuel consumption directly. The input `data:mission:MTOW_mission:taxi_out:fuel` should be set to 276kg.

Weights should also be checked to be in accordance with the mission specifications: 

* `data:weight:aircraft:max_payload` = 19608kg
* `data:weight:aircraft:payload` = 17000kg


In [ ]:
input_file = oad.generate_inputs(OAD_CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

In [ ]:
oad.variable_viewer(input_file)

### 1.4. Run the OAD process

Now, run the problem and check the output file:

In [ ]:
MTOW_problem = oad.evaluate_problem(OAD_CONFIGURATION_FILE, overwrite=True)

### 1.5. Results Analysis

You can now print the output file and compare the results with the CeRAS report ones.

In [ ]:
oad.variable_viewer(MTOW_problem.output_file_path)

Particularly, the Maximum Take Off Weight (MTOW) of CeRAS report is 77000kg. Here you should find 77006kg for the `data:mission:MTOW_mission:TOW` and for `data:weight:aircraft:MTOW`. In the same way, the reference mission fuel is 18183kg, you should have 18368kg for `data:mission:MTOW_mission:block_fuel`.

The Operating Empty Weight (OWE) is also assessed as `data:weight:aircraft:MTOW` and estimated to 42000kg, where the reference value is 42082kg.

A more practical way is to use the visualisation tools that were presented in a previous tutorial to plot the mass breakdown and the aircraft geometry and compare them with the CeRAS reference aircraft data. 

In [ ]:
oad.mass_breakdown_sun_plot(MTOW_problem.output_file_path)

In [ ]:
fig = oad.mass_breakdown_bar_plot(MTOW_problem.output_file_path, name="FAST-OAD")
fig = oad.mass_breakdown_bar_plot(REFERENCE_FILE_FOR_PLOT, name="CeRAS Report", fig=fig)
fig.show()

**Remark:** Difference are observed between *crew*, *systems* and *operational* weight categories. That comes from the different mass breakdown strategies between FAST-OAD and CeRAS team. For example, the CeRAS team included crew weight within the *operational* weights while FAST-OAD has a dedicated category. On the contrary the *operational* category does not exist in FAST-OAD and his contents are integrated within the *systems* weights. For more details about the mass breakdown strategy and the content of each category:
* FAST-OAD strategy: please refer to the sunburst plot here above where the detail of each category and the associated weights are plotted
* CeRAS strategy: please refer to CeRAS web page [here](https://ceras.ilr.rwth-aachen.de/trac/wiki/CeRAS/Definitions/MassBreakdownDefinition)

In [ ]:
fig = oad.wing_geometry_plot(MTOW_problem.output_file_path, name="FAST-OAD")
fig = oad.wing_geometry_plot(REFERENCE_FILE_FOR_PLOT, name="CeRAS Report", fig=fig)
fig.show()

In [ ]:
oad.aircraft_geometry_plot(MTOW_problem.output_file_path)

You can also plot the mission profile and fuel consumption history to compare with CeRAS ones: 

In [ ]:
plot_mission_against_ceras(
    pth.join(WORK_FOLDER_PATH, "oad_sizing.csv"), CERAS_REFERENCE_RESULTS_FILE
)

***
***

## 2. Operational Missions
Starting from your CeRAS design you will now discover how to run missions analyses for the two operational mission defined in the CeRAS report. 

The first mission is the Standard Passenger Payload desing mission whose characteristics are: 

- Range = 2750NM
- Diversion range = 200NM
- Payload = 13608kg

The second mission is the Standard Passenger Payload study mission whose characteristics are: 

- Range = 500NM
- Diversion range = 200NM
- Payload = 13608kg

For those two missions the objective is to retrieve the fuel weight and consequently the take-off weight found in the CeRAS report.  

The two missions profiles are described in the same [mission file](./data/operational_missions.yml). They are composed of a main route and a diversion. Only the range of the flight will be different and change in the input file. 


### 2.1. Updating the configuration file 
Once you create an adapted mission file you should generate or retrieve a configuration file that would allow for a mission analysis to be run. Let's have a look to such a [file](./data/operational_missions.yml): 

In [ ]:
CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, "operational_missions.yml")

The [configuration file](./data/operational_missions.yml) shows that high-speed aerodynamics subgroup and performances modules are called. The low-speed polar is already available in the [reference data](./data/CeRAS_reference_data.xml) file and you need high-speed aerodynamics subgroup to compute cruise one.

**Important note:** Notice that the two missions are run successively through two performance group definition: the first one is `model.SPP_design` the second one `model.SPP_study`. Of course, you can separate the two analyses in two different configuration files. 

The option `adjust_fuel` that is set to **True** indicates that fuel weight is adjusted to the need of the mission. The take-off weight will be the results of an iterative procedure and is computed as follow: 

$ TOW = OWE + PL + BF - F_{taxi-out} - F_{TO\ run} $ with $OWE$ from the OAD sizing results, $BF$ being the block (loaded) fuel, $F_{taxi-out}$ and $F_{TO\ run}$ being respectively the consumed fuel during taxi-out and takeoff run phases. 

You need to set the option `add_solver` to **True** to allow for the iterative process to take place if no solvers are defined in the configuration file. 

Note again that the `mission_name` option must correspond to a mission described in the [mission file](./data/CeRAS_missions.yml).  

### 2.2. The input file

As indicated in the introduction of this part, you will start from the results of the OAD sizing process you have just run to compute the operational missions take-off weight and fuel consumption. 

To do so, you just have to use the XML [output file](./data/oad_sizing_out.xml) from the OAD process as the source file to generate the new input file: 

In [ ]:
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "oad_sizing_out.xml")
input_file = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

**Important note:** You should check carefully the mission inputs to be sure they are consistent with the mission specifications and that no "nan" values are present. Indeed, the file we provided as `SOURCE_FILE` results from the same OAD process you have just run but we added the correct mission specifications to enable you to run the computation directly without having to input the correct values. However, if you choose to use directly the results from your OAD run calling `problem.output_file_path` as source file in `generate_inputs` arguments you *will have* to update the values for the mission specifications one by one.  

In [ ]:
oad.variable_viewer(input_file)

### 2.3. Operational Missions Runs

You can now run the missions analyses:

In [ ]:
SPP_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

The XML [output file](./workdir/operational_missions_out.xml) can be displayed in a table format using `variable_viewer`function. And you can compare the fuel consumptions and TOW with the expected values from CeRAS report for the two missions. 

In [ ]:
oad.variable_viewer(SPP_problem.output_file_path)
print(
    "Mission fuel SPP Design: ",
    SPP_problem["data:mission:SPP_design:needed_block_fuel"],
    "kg --",
    "Expected value: 18678kg",
)
print(
    "Mission fuel SPP Study: ",
    SPP_problem["data:mission:SPP_study:needed_block_fuel"],
    "kg --",
    "Expected value: 6665kg",
)

print(
    "TOW SPP Design: ",
    SPP_problem["data:mission:SPP_design:TOW"],
    "kg --",
    "Expected value: 74102kg",
)
print(
    "TOW SPP Study: ", SPP_problem["data:mission:SPP_study:TOW"], "kg --", "Expected value: 62089kg"
)

And you can compare the mission profile and fuel consumption history with CeRAS data. 

In [ ]:
plot_mission_against_ceras(
    pth.join(WORK_FOLDER_PATH, "SPP_design.csv"), CERAS_SPP_DESIGN_REFERENCE_RESULTS_FILE
)

In [ ]:
plot_mission_against_ceras(
    pth.join(WORK_FOLDER_PATH, "SPP_study.csv"), CERAS_SPP_STUDY_REFERENCE_RESULTS_FILE
)